In [58]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import os
import vectorbt as vbt
import io
import sys
from contextlib import redirect_stdout
from datetime import datetime
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')





In [59]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple, List, Union, Optional
from datetime import datetime
from xbbg import blp

# Define default mappings outside the class as constants
DEFAULT_OHLC_MAPPING = {
    ('I05510CA Index', 'INDEX_OAS_TSY_BP'): 'cad_oas',
    ('LF98TRUU Index', 'INDEX_OAS_TSY_BP'): 'us_hy_oas',
    ('LUACTRUU Index', 'INDEX_OAS_TSY_BP'): 'us_ig_oas',
    ('SPTSX Index', 'PX_LAST'): 'tsx',
    ('VIX Index', 'PX_LAST'): 'vix',
    ('USYC3M30 Index', 'PX_LAST'): 'us_3m_10y',
    ('BCMPUSGR Index', 'PX_LAST'): 'us_growth_surprises',
    ('BCMPUSIF Index', 'PX_LAST'): 'us_inflation_surprises',
    ('LEI YOY  Index', 'PX_LAST'): 'us_lei_yoy',
    ('.HARDATA G Index', 'PX_LAST'): 'us_hard_data_surprises',
    ('CGERGLOB Index', 'PX_LAST'): 'us_equity_revisions',
    ('.ECONREGI G Index', 'PX_LAST'): 'us_economic_regime',
}

DEFAULT_ER_YTD_MAPPING = {
    ('I05510CA Index', 'INDEX_EXCESS_RETURN_YTD'): 'cad_ig_er',
    ('LF98TRUU Index', 'INDEX_EXCESS_RETURN_YTD'): 'us_hy_er',
    ('LUACTRUU Index', 'INDEX_EXCESS_RETURN_YTD'): 'us_ig_er',
}

class DataFetcher:
    def __init__(
        self,
        start_date: str = '2002-01-01',
        end_date: str = None,
        periodicity: str = 'D',
        align_start: bool = True,
        fill: str = 'ffill',
        start_date_align: str = 'yes',
        ohlc_mapping: Dict[Tuple[str, str], str] = None,
        er_ytd_mapping: Dict[Tuple[str, str], str] = None
    ):
        """
        Initialize the DataFetcher class with all configuration parameters

        Args:
            start_date: Start date in YYYY-MM-DD format
            end_date: End date in YYYY-MM-DD format (defaults to current date)
            periodicity: Data frequency ('D' for daily)
            align_start: Whether to align data from the start date
            fill: Fill method ('ffill' for forward fill)
            start_date_align: Whether to align start dates ('yes' or 'no')
            ohlc_mapping: Custom mapping for price data (defaults to predefined mapping)
            er_ytd_mapping: Custom mapping for excess return data (defaults to predefined mapping)
        """
        # Set default end date to today if not provided
        if end_date is None:
            self.end_date = datetime.now().strftime('%Y-%m-%d')
        else:
            self.end_date = end_date

        # Store all parameters as instance attributes
        self.start_date = start_date
        self.periodicity = periodicity
        self.align_start = align_start
        self.fill = fill
        self.start_date_align = start_date_align

        # Use provided mappings or default to the ones defined outside the class
        self.ohlc_mapping = ohlc_mapping if ohlc_mapping is not None else DEFAULT_OHLC_MAPPING
        self.er_ytd_mapping = er_ytd_mapping if er_ytd_mapping is not None else DEFAULT_ER_YTD_MAPPING

        # List of problematic dates that need cleaning
        self.bad_dates = {
            '2005-11-15': {'column': 'cad_oas', 'action': 'use_previous'}
        }

    def update_parameters(self, **kwargs):
        """
        Update any of the class parameters

        Args:
            **kwargs: Any parameter to update
        """
        for key, value in kwargs.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                raise AttributeError(f"DataFetcher has no attribute '{key}'")

        # If end_date is updated to None, set it to current date
        if 'end_date' in kwargs and kwargs['end_date'] is None:
            self.end_date = datetime.now().strftime('%Y-%m-%d')

    def fetch_bloomberg_data(self, mapping: Optional[Dict[Tuple[str, str], str]] = None) -> pd.DataFrame:
        """
        Fetch data from Bloomberg using xbbg using class parameters

        Args:
            mapping: Optional override for the mapping to use

        Returns:
            DataFrame with requested data
        """
        # Use provided mapping or default to ohlc_mapping
        mapping_to_use = mapping if mapping is not None else self.ohlc_mapping

        securities = list(set(security for security, _ in mapping_to_use.keys()))
        fields = list(set(field for _, field in mapping_to_use.keys()))

        # Fetch data using xbbg
        df = blp.bdh(
            tickers=securities,
            flds=fields,
            start_date=self.start_date,
            end_date=self.end_date,
            Per=self.periodicity
        )

        # Create a new DataFrame with renamed columns
        renamed_df = pd.DataFrame(index=df.index)
        for (security, field), new_name in mapping_to_use.items():
            if (security, field) in df.columns:
                renamed_df[new_name] = df[(security, field)]

        return renamed_df

    def convert_er_ytd_to_index(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Convert excess return YTD data to an index, only for securities in er_ytd_mapping

        Args:
            df: DataFrame containing excess return YTD columns

        Returns:
            DataFrame with excess return columns converted to indices
        """
        result = pd.DataFrame(index=df.index)

        # Only convert columns that are in the er_ytd_mapping values
        er_columns = list(self.er_ytd_mapping.values())
        for column in df.columns:
            if column in er_columns:
                # Convert YTD returns to daily returns
                daily_returns = df[column].diff()

                # Create index starting at 100
                index_values = (1 + daily_returns / 100).cumprod() * 100
                result[f"{column}_index"] = index_values

        return result

    def merge_dfs(self, df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
        """
        Merge two DataFrames with proper date alignment and filling using class parameters

        Args:
            df1: First DataFrame
            df2: Second DataFrame

        Returns:
            Merged DataFrame
        """
        # Merge DataFrames
        merged = pd.concat([df1, df2], axis=1)

        # Fill missing values
        if self.fill:
            merged = merged.fillna(method=self.fill)

        return merged

    def clean_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Clean specific known data issues defined in bad_dates

        Args:
            df: DataFrame to clean

        Returns:
            Cleaned DataFrame
        """
        # Make a copy to avoid modification warnings
        cleaned_df = df.copy()

        # Process each bad date according to defined actions
        for date, info in self.bad_dates.items():
            if date in cleaned_df.index and info['column'] in cleaned_df.columns:
                if info['action'] == 'use_previous':
                    prev_value = cleaned_df.loc[cleaned_df.index < date, info['column']].iloc[-1]
                    cleaned_df.loc[date, info['column']] = prev_value

        return cleaned_df

    def get_full_dataset(self) -> pd.DataFrame:
        """
        Get a complete dataset with both price data and excess return indices using class parameters.
        If start_date_align is 'yes', will find the first date where all data is available.

        Returns:
            Complete DataFrame with all requested data
        """
        # Fetch the main price data
        df_ohlc = self.fetch_bloomberg_data(mapping=self.ohlc_mapping)

        # Fetch the excess return YTD data
        er_ytd_df = self.fetch_bloomberg_data(mapping=self.er_ytd_mapping)

        # Convert excess return YTD to index (only for columns in er_ytd_mapping)
        er_index_df = self.convert_er_ytd_to_index(er_ytd_df)

        # Merge all the datasets
        final_df = self.merge_dfs(df_ohlc, er_index_df)

        # Clean any known data issues
        final_df = self.clean_data(final_df)

        # If start_date_align is 'yes', keep only rows where all data is available
        if self.start_date_align == 'yes':
            # Find the first date with no NaN values
            non_null_df = final_df.dropna(how='any')
            if not non_null_df.empty:
                first_complete_date = non_null_df.index[0]
                # Filter to only include dates on or after the first complete date
                final_df = final_df[final_df.index >= first_complete_date]

        # Apply any final fill operations specified
        if self.fill:
            final_df = final_df.fillna(method=self.fill)

        return final_df

In [60]:
# Test with a specific date range
data_fetcher = DataFetcher(
    start_date='2010-01-01',
    end_date='2015-12-31',
    start_date_align='yes'
)
date_range_df = data_fetcher.get_full_dataset()
print(date_range_df.info())
print(f"Date range: {date_range_df.index.min()} to {date_range_df.index.max()}")

<class 'pandas.core.frame.DataFrame'>
Index: 1565 entries, 2010-01-31 to 2015-12-31
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cad_oas                 1565 non-null   float64
 1   us_hy_oas               1565 non-null   float64
 2   us_ig_oas               1565 non-null   float64
 3   tsx                     1565 non-null   float64
 4   vix                     1565 non-null   float64
 5   us_3m_10y               1565 non-null   float64
 6   us_growth_surprises     1565 non-null   float64
 7   us_inflation_surprises  1565 non-null   float64
 8   us_lei_yoy              1565 non-null   float64
 9   us_hard_data_surprises  1565 non-null   float64
 10  us_equity_revisions     1565 non-null   float64
 11  us_economic_regime      1565 non-null   float64
 12  cad_ig_er_index         1565 non-null   float64
 13  us_hy_er_index          1565 non-null   float64
 14  us_ig_er_index          1565 n

In [61]:
# Create with default parameters then update
data_fetcher = DataFetcher()
data_fetcher.update_parameters(
    start_date='2020-01-01',
    end_date='2021-12-31',
    start_date_align='no'
)
updated_df = data_fetcher.get_full_dataset()
print(updated_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 530 entries, 2020-01-01 to 2021-12-31
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cad_oas                 529 non-null    float64
 1   us_hy_oas               529 non-null    float64
 2   us_ig_oas               529 non-null    float64
 3   tsx                     529 non-null    float64
 4   vix                     529 non-null    float64
 5   us_3m_10y               530 non-null    float64
 6   us_growth_surprises     530 non-null    float64
 7   us_inflation_surprises  530 non-null    float64
 8   us_lei_yoy              508 non-null    float64
 9   us_hard_data_surprises  519 non-null    float64
 10  us_equity_revisions     528 non-null    float64
 11  us_economic_regime      508 non-null    float64
 12  cad_ig_er_index         528 non-null    float64
 13  us_hy_er_index          528 non-null    float64
 14  us_ig_er_index          528 non

In [113]:
# Define date range and periodicity
start_date = '2003-01-01'
end_date = (datetime.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
periodicity = 'DAILY'  # Change this to 'WEEKLY', 'MONTHLY', etc. as needed

# Canadian OAS
cad_oas_data = blp.bdh('I05510CA Index', 'INDEX_OAS_TSY_BP', start_date, end_date, periodicitySelection=periodicity)
cad_oas = pd.Series(cad_oas_data.values.flatten(), index=cad_oas_data.index, name='cad_oas')

# US High Yield OAS
us_hy_oas_data = blp.bdh('LF98TRUU Index', 'INDEX_OAS_TSY_BP', start_date, end_date, periodicitySelection=periodicity)
us_hy_oas = pd.Series(us_hy_oas_data.values.flatten(), index=us_hy_oas_data.index, name='us_hy_oas')

# US Investment Grade OAS
us_ig_oas_data = blp.bdh('LUACTRUU Index', 'INDEX_OAS_TSY_BP', start_date, end_date, periodicitySelection=periodicity)
us_ig_oas = pd.Series(us_ig_oas_data.values.flatten(), index=us_ig_oas_data.index, name='us_ig_oas')

# TSX Index
tsx_data = blp.bdh('SPTSX Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
tsx = pd.Series(tsx_data.values.flatten(), index=tsx_data.index, name='tsx')

# VIX Index
vix_data = blp.bdh('VIX Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
vix = pd.Series(vix_data.values.flatten(), index=vix_data.index, name='vix')

# US 3M-10Y Yield Curve
us_3m_10y_data = blp.bdh('USYC3M30 Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_3m_10y = pd.Series(us_3m_10y_data.values.flatten(), index=us_3m_10y_data.index, name='us_3m_10y')

# US Growth Surprises
us_growth_surprises_data = blp.bdh('BCMPUSGR Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_growth_surprises = pd.Series(us_growth_surprises_data.values.flatten(), index=us_growth_surprises_data.index, name='us_growth_surprises')

# US Inflation Surprises
us_inflation_surprises_data = blp.bdh('BCMPUSIF Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_inflation_surprises = pd.Series(us_inflation_surprises_data.values.flatten(), index=us_inflation_surprises_data.index, name='us_inflation_surprises')

# US Leading Economic Indicators YoY
us_lei_yoy_data = blp.bdh('LEI YOY  Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_lei_yoy = pd.Series(us_lei_yoy_data.values.flatten(), index=us_lei_yoy_data.index, name='us_lei_yoy')

# US Hard Data Surprises
us_hard_data_surprises_data = blp.bdh('.HARDATA G Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_hard_data_surprises = pd.Series(us_hard_data_surprises_data.values.flatten(), index=us_hard_data_surprises_data.index, name='us_hard_data_surprises')

# US Equity Revisions
us_equity_revisions_data = blp.bdh('CGERGLOB Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_equity_revisions = pd.Series(us_equity_revisions_data.values.flatten(), index=us_equity_revisions_data.index, name='us_equity_revisions')

# US Economic Regime
us_economic_regime_data = blp.bdh('.ECONREGI G Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_economic_regime = pd.Series(us_economic_regime_data.values.flatten(), index=us_economic_regime_data.index, name='us_economic_regime')

# SPX 1yrBF EPS (Consensus)
spx_1bf_eps_data = blp.bdh('SPX Index','BEST_EPS', start_date, end_date, BEST_PERIOD_OVERRIDE='1BF')
spx_1bf_eps= pd.Series(spx_1bf_eps_data.values.flatten(), index=spx_1bf_eps_data.index, name='spx_1bf_eps')

# Combine all series into a single DataFrame using pd.concat
df = pd.concat([
    cad_oas,
    us_hy_oas,
    us_ig_oas,
    tsx,
    vix,
    us_3m_10y,
    us_growth_surprises,
    us_inflation_surprises,
    us_lei_yoy,
    us_hard_data_surprises,
    us_equity_revisions,
    us_economic_regime,
    spx_1bf_eps
], axis=1)

# Handle missing values (forward fill is common in financial time series)
df = df.fillna(method='ffill').dropna()

# Ensure we only have appropriate frequency days based on periodicity
if periodicity == 'DAILY':
    df = df.asfreq('B')  # Business days
elif periodicity == 'WEEKLY':
    df = df.asfreq('W-FRI')  # Weekly on Friday
elif periodicity == 'MONTHLY':
    df = df.asfreq('M')  # Month end

# Final check for future dates
today = pd.Timestamp.today().normalize()
if any(df.index > today):
    print(f"Warning: DataFrame contains future dates")
    df = df[df.index <= today]

# Print date range information
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Number of data points: {len(df)}")
df

# Define date range and periodicity
start_date = '2003-01-01'
end_date = (datetime.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
periodicity = 'DAILY'  # Change this to 'WEEKLY', 'MONTHLY', etc. as needed

# Canadian OAS
cad_oas_data = blp.bdh('I05510CA Index', 'INDEX_OAS_TSY_BP', start_date, end_date, periodicitySelection=periodicity)
cad_oas = pd.Series(cad_oas_data.values.flatten(), index=cad_oas_data.index, name='cad_oas')

# US High Yield OAS
us_hy_oas_data = blp.bdh('LF98TRUU Index', 'INDEX_OAS_TSY_BP', start_date, end_date, periodicitySelection=periodicity)
us_hy_oas = pd.Series(us_hy_oas_data.values.flatten(), index=us_hy_oas_data.index, name='us_hy_oas')

# US Investment Grade OAS
us_ig_oas_data = blp.bdh('LUACTRUU Index', 'INDEX_OAS_TSY_BP', start_date, end_date, periodicitySelection=periodicity)
us_ig_oas = pd.Series(us_ig_oas_data.values.flatten(), index=us_ig_oas_data.index, name='us_ig_oas')

# TSX Index
tsx_data = blp.bdh('SPTSX Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
tsx = pd.Series(tsx_data.values.flatten(), index=tsx_data.index, name='tsx')

# VIX Index
vix_data = blp.bdh('VIX Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
vix = pd.Series(vix_data.values.flatten(), index=vix_data.index, name='vix')

# US 3M-10Y Yield Curve
us_3m_10y_data = blp.bdh('USYC3M30 Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_3m_10y = pd.Series(us_3m_10y_data.values.flatten(), index=us_3m_10y_data.index, name='us_3m_10y')

# US Growth Surprises
us_growth_surprises_data = blp.bdh('BCMPUSGR Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_growth_surprises = pd.Series(us_growth_surprises_data.values.flatten(), index=us_growth_surprises_data.index, name='us_growth_surprises')

# US Inflation Surprises
us_inflation_surprises_data = blp.bdh('BCMPUSIF Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_inflation_surprises = pd.Series(us_inflation_surprises_data.values.flatten(), index=us_inflation_surprises_data.index, name='us_inflation_surprises')

# US Leading Economic Indicators YoY
us_lei_yoy_data = blp.bdh('LEI YOY  Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_lei_yoy = pd.Series(us_lei_yoy_data.values.flatten(), index=us_lei_yoy_data.index, name='us_lei_yoy')

# US Hard Data Surprises
us_hard_data_surprises_data = blp.bdh('.HARDATA G Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_hard_data_surprises = pd.Series(us_hard_data_surprises_data.values.flatten(), index=us_hard_data_surprises_data.index, name='us_hard_data_surprises')

# US Equity Revisions
us_equity_revisions_data = blp.bdh('CGERGLOB Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_equity_revisions = pd.Series(us_equity_revisions_data.values.flatten(), index=us_equity_revisions_data.index, name='us_equity_revisions')

# US Economic Regime
us_economic_regime_data = blp.bdh('.ECONREGI G Index', 'PX_LAST', start_date, end_date, periodicitySelection=periodicity)
us_economic_regime = pd.Series(us_economic_regime_data.values.flatten(), index=us_economic_regime_data.index, name='us_economic_regime')

# SPX 1yrBF EPS (Consensus)
spx_1bf_eps_data = blp.bdh('SPX Index', 'BEST_EPS', start_date, end_date, BEST_FPERIOD_OVERRIDE='1BF')
spx_1bf_eps= pd.Series(spx_1bf_eps_data.values.flatten(), index=spx_1bf_eps_data.index, name='spx_1bf_eps')

# SPX 1yrBF Sales (Consensus)
spx_1bf_sales_data = blp.bdh('SPX Index', 'BEST_SALES', start_date, end_date, BEST_FPERIOD_OVERRIDE='1BF')
spx_1bf_sales= pd.Series(spx_1bf_sales_data.values.flatten(), index=spx_1bf_sales_data.index, name='spx_1bf_sales')

# TSX 1yrBF EPS (Consensus)
tsx_1bf_eps_data = blp.bdh('SPTSX Index', 'BEST_EPS', start_date, end_date, BEST_FPERIOD_OVERRIDE='1BF')
tsx_1bf_eps= pd.Series(tsx_1bf_eps_data.values.flatten(), index=tsx_1bf_eps_data.index, name='tsx_1bf_eps')

# TSX 1yrBF Sales (Consensus)
tsx_1bf_sales_data = blp.bdh('SPTSX Index', 'BEST_SALES', start_date, end_date, BEST_FPERIOD_OVERRIDE='1BF')
tsx_1bf_sales = pd.Series(tsx_1bf_sales_data.values.flatten(), index=tsx_1bf_sales_data.index, name='tsx_1bf_sales')

# Combine all series into a single DataFrame using pd.concat
df = pd.concat([
    cad_oas,
    us_hy_oas,
    us_ig_oas,
    tsx,
    vix,
    us_3m_10y,
    us_growth_surprises,
    us_inflation_surprises,
    us_lei_yoy,
    us_hard_data_surprises,
    us_equity_revisions,
    us_economic_regime,
    spx_1bf_eps,
    spx_1bf_sales,
    tsx_1bf_eps,
    tsx_1bf_sales
], axis=1)


# Ensure we only have appropriate frequency days based on periodicity
if periodicity == 'DAILY':
    df = df.asfreq('B')  # Business days
elif periodicity == 'WEEKLY':
    df = df.asfreq('W-FRI')  # Weekly on Friday
elif periodicity == 'MONTHLY':
    df = df.asfreq('M')  # Month end

# Final check for future dates
today = pd.Timestamp.today().normalize()
if any(df.index > today):
    print(f"Warning: DataFrame contains future dates")
    df = df[df.index <= today]

# Print date range information
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Number of data points: {len(df)}")

# Handle missing values (forward fill is common in financial time series)
df = df.fillna(method='ffill').dropna()
df

Date range: 2003-01-01 00:00:00 to 2025-03-13 00:00:00
Number of data points: 5792
Date range: 2003-01-01 00:00:00 to 2025-03-13 00:00:00
Number of data points: 5792


,cad_oas,us_hy_oas,us_ig_oas,tsx,vix,us_3m_10y,us_growth_surprises,us_inflation_surprises,us_lei_yoy,us_hard_data_surprises,us_equity_revisions,us_economic_regime,spx_1bf_eps,spx_1bf_sales,tsx_1bf_eps,tsx_1bf_sales
2003-11-18,80.250545,442.148787,100.708833,7737.39,19.11,405.524,0.36977,-0.17776,5.7,0.3537,0.10,1.00,61.4777,675.7230,560.4854,6378.6926
2003-11-19,82.801336,434.467445,100.860500,7801.09,18.80,413.633,0.40873,-0.17687,5.7,0.4087,0.10,1.00,61.4579,675.7107,560.9816,6389.3538
2003-11-20,74.367794,441.315940,102.127192,7809.78,19.48,406.952,0.42674,-0.17598,5.7,0.4209,0.10,1.00,61.4510,675.8926,560.1786,6395.5928
2003-11-21,73.317473,439.114196,101.437013,7783.59,18.98,408.332,0.42460,-0.17510,5.7,0.4209,0.11,1.00,61.5070,676.1503,560.0390,6406.0664
2003-11-24,84.962356,430.098810,100.931427,7850.15,17.44,412.792,0.42248,-0.17423,5.7,0.4209,0.11,1.00,61.5620,676.1802,561.3624,6374.1137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-07,110.630700,290.402100,86.545300,24758.76,23.37,29.060,-0.40710,0.60899,-2.6,-0.0041,-0.06,0.38,275.0860,2002.3156,1675.7925,12763.7324
2025-03-10,111.791600,309.179100,90.068000,24380.71,27.86,25.180,-0.40507,0.60594,-2.6,-0.0041,-0.06,0.38,275.3340,2003.2743,1677.4366,12652.4509
2025-03-11,112.961600,315.272500,93.114400,24248.20,26.92,30.069,-0.39919,0.60291,-2.6,-0.0041,-0.06,0.38,275.3361,2003.5016,1676.5041,12665.5151
2025-03-12,112.401500,311.726500,93.220000,24423.34,24.23,32.400,-0.39695,0.48326,-2.6,-0.0041,-0.06,0.38,275.3556,2003.9729,1676.4100,12651.7278


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5563 entries, 2003-11-18 to 2025-03-13
Freq: B
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cad_oas                 5563 non-null   float64
 1   us_hy_oas               5563 non-null   float64
 2   us_ig_oas               5563 non-null   float64
 3   tsx                     5563 non-null   float64
 4   vix                     5563 non-null   float64
 5   us_3m_10y               5563 non-null   float64
 6   us_growth_surprises     5563 non-null   float64
 7   us_inflation_surprises  5563 non-null   float64
 8   us_lei_yoy              5563 non-null   float64
 9   us_hard_data_surprises  5563 non-null   float64
 10  us_equity_revisions     5563 non-null   float64
 11  us_economic_regime      5563 non-null   float64
 12  spx_1bf_eps             5563 non-null   float64
 13  spx_1bf_sales           5563 non-null   float64
 14  tsx_1bf_eps   

In [116]:
# Calculate percentage changes for selected columns
# Define lookback periods based on frequency
if periodicity == 'DAILY':
    short_period = 30    # ~1 month
    medium_period = 90   # ~3 months
    long_period = 265    # ~1 year (trading days)
    period_suffix = 'd'
elif periodicity == 'WEEKLY':
    short_period = 4     # ~1 month
    medium_period = 13   # ~3 months
    long_period = 52     # ~1 year
    period_suffix = 'w'
elif periodicity == 'MONTHLY':
    short_period = 1     # 1 month
    medium_period = 3    # 3 months
    long_period = 12     # 1 year
    period_suffix = 'm'
else:
    short_period = 30
    medium_period = 90
    long_period = 265
    period_suffix = 'd'

# Calculate percentage changes for TSX
df[f'tsx_{short_period}{period_suffix}_pct'] = df['tsx'].pct_change(periods=short_period) * 100
df[f'tsx_{medium_period}{period_suffix}_pct'] = df['tsx'].pct_change(periods=medium_period) * 100
df[f'tsx_{long_period}{period_suffix}_pct'] = df['tsx'].pct_change(periods=long_period) * 100

# Calculate percentage changes for SPX forward EPS
df[f'spx_1bf_eps_{short_period}{period_suffix}_pct'] = df['spx_1bf_eps'].pct_change(periods=short_period) * 100
df[f'spx_1bf_eps_{medium_period}{period_suffix}_pct'] = df['spx_1bf_eps'].pct_change(periods=medium_period) * 100
df[f'spx_1bf_eps_{long_period}{period_suffix}_pct'] = df['spx_1bf_eps'].pct_change(periods=long_period) * 100

# Calculate percentage changes for SPX forward Sales
df[f'spx_1bf_sales_{short_period}{period_suffix}_pct'] = df['spx_1bf_sales'].pct_change(periods=short_period) * 100
df[f'spx_1bf_sales_{medium_period}{period_suffix}_pct'] = df['spx_1bf_sales'].pct_change(periods=medium_period) * 100
df[f'spx_1bf_sales_{long_period}{period_suffix}_pct'] = df['spx_1bf_sales'].pct_change(periods=long_period) * 100

# Calculate percentage changes for TSX forward EPS
df[f'tsx_1bf_eps_{short_period}{period_suffix}_pct'] = df['tsx_1bf_eps'].pct_change(periods=short_period) * 100
df[f'tsx_1bf_eps_{medium_period}{period_suffix}_pct'] = df['tsx_1bf_eps'].pct_change(periods=medium_period) * 100
df[f'tsx_1bf_eps_{long_period}{period_suffix}_pct'] = df['tsx_1bf_eps'].pct_change(periods=long_period) * 100

# Drop rows with NaN values from the percentage calculations
# This will remove the first 'long_period' rows since they don't have enough history
df= df.dropna()

# Display information about the DataFrame with percentage changes
print(f"DataFrame with percentage changes:")
print(f"Date range: {df_with_pct.index.min()} to {df_with_pct.index.max()}")
print(f"Number of data points: {len(df_with_pct)}")
print(f"Number of columns: {len(df_with_pct.columns)}")

# Display the first few rows of the DataFrame with percentage changes
df

DataFrame with percentage changes:
Date range: 2004-11-23 00:00:00 to 2025-03-13 00:00:00
Number of data points: 5298
Number of columns: 28


,cad_oas,us_hy_oas,us_ig_oas,tsx,vix,us_3m_10y,us_growth_surprises,us_inflation_surprises,us_lei_yoy,us_hard_data_surprises,...,tsx_265d_pct,spx_1bf_eps_30d_pct,spx_1bf_eps_90d_pct,spx_1bf_eps_265d_pct,spx_1bf_sales_30d_pct,spx_1bf_sales_90d_pct,spx_1bf_sales_265d_pct,tsx_1bf_eps_30d_pct,tsx_1bf_eps_90d_pct,tsx_1bf_eps_265d_pct
2004-11-23,65.157982,306.681386,83.954184,8995.19,12.67,266.345,0.20854,0.10390,11.1,-0.0629,...,16.256128,2.018987,5.377233,23.148394,0.757357,4.006474,9.771933,0.000000,0.000000,0.958544
2004-11-24,65.013946,303.642533,83.249958,8976.09,12.72,267.451,0.16996,0.10338,11.1,-0.0410,...,15.061998,2.078750,5.346263,23.288137,0.733092,3.880217,9.853980,0.000000,0.000000,0.869244
2004-11-25,65.013946,303.642533,83.249958,9025.90,12.72,267.451,0.16911,0.10286,11.1,-0.0410,...,15.571757,2.360013,5.411752,23.375372,0.764794,3.822230,9.837273,0.000000,0.000000,1.013837
2004-11-26,64.518796,300.123821,82.486817,9057.97,12.78,269.219,0.16827,0.10235,11.1,-0.0410,...,16.372651,2.296174,5.353009,23.279139,0.797737,3.798510,9.784526,0.000000,0.000000,1.039017
2004-11-29,63.746985,297.330883,83.305442,9056.41,13.30,276.414,0.16743,0.10184,11.1,-0.0410,...,15.366076,2.034406,5.344008,23.157792,0.607910,3.771529,9.684208,0.000000,0.000000,0.800820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-07,110.630700,290.402100,86.545300,24758.76,23.37,29.060,-0.40710,0.60899,-2.6,-0.0041,...,14.877310,0.372429,3.564128,12.077437,0.653248,1.858290,5.782820,1.306661,4.179430,13.540147
2025-03-10,111.791600,309.179100,90.068000,24380.71,27.86,25.180,-0.40507,0.60594,-2.6,-0.0041,...,13.235013,0.433859,3.464486,12.103848,0.657865,1.888131,5.802669,1.274226,4.186188,13.502942
2025-03-11,112.961600,315.272500,93.114400,24248.20,26.92,30.069,-0.39919,0.60291,-2.6,-0.0041,...,12.646469,0.409755,3.453690,12.081201,0.638839,1.925539,5.800001,1.243043,4.161886,13.527133
2025-03-12,112.401500,311.726500,93.220000,24423.34,24.23,32.400,-0.39695,0.48326,-2.6,-0.0041,...,13.102645,0.515288,3.445703,12.050551,0.666108,1.909673,5.815741,1.173513,4.103235,13.578390
